In [1]:
import os
import sys
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# 파일 경로 설정 (현재 폴더 내에 abalone.csv 파일이 있어야 함)
file_path = "abalone.csv"

if not os.path.exists(file_path):
    print(f"파일을 찾을 수 없습니다: {file_path}\n현재 작업 디렉토리: {os.getcwd()}")
    print("파일이 현재 경로에 있는지 확인하거나, file_path를 수정하세요.")
    sys.exit()  # 파일을 찾을 수 없으면 프로그램 종료

# 1. 데이터 로드 및 전처리
df = pd.read_csv(file_path)
# abalone.csv는 보통 'Sex, Length, Diameter, Height, Whole weight, Shucked weight, Viscera weight, Shell weight, Rings' 컬럼을 가집니다.
# 여기서는 Rings 값을 타겟으로 사용합니다.
# 범주형 변수인 'Sex'를 원-핫 인코딩합니다.
df = pd.get_dummies(df, columns=["Sex"])

X = df.drop(columns=["Rings"]).values.astype(np.float32)
y = df["Rings"].values.astype(np.float32).reshape(-1, 1)

# 2. 학습/테스트 데이터 분리 및 스케일링
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler_reg = StandardScaler()
X_train = scaler_reg.fit_transform(X_train)
X_test = scaler_reg.transform(X_test)

# numpy 배열 -> PyTorch 텐서 변환
X_train_tensor = torch.tensor(X_train)
X_test_tensor = torch.tensor(X_test)
y_train_tensor = torch.tensor(y_train)
y_test_tensor = torch.tensor(y_test)

# 3. 회귀용 신경망 모델 정의 (간단한 3층 신경망)
class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # Rings 예측: 연속형 값
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

input_dim = X_train_tensor.shape[1]
model_reg = RegressionModel(input_dim)

# 4. 손실함수 및 옵티마이저 설정 (MSELoss 사용)
criterion_reg = nn.MSELoss()
optimizer_reg = optim.Adam(model_reg.parameters(), lr=0.001)

# 5. 모델 학습 (100 에포크)
epochs_reg = 100
for epoch in range(epochs_reg):
    model_reg.train()
    optimizer_reg.zero_grad()
    outputs_reg = model_reg(X_train_tensor)
    loss_reg = criterion_reg(outputs_reg, y_train_tensor)
    loss_reg.backward()
    optimizer_reg.step()
    if (epoch+1) % 10 == 0:
        print(f"[Regression] Epoch {epoch+1}/{epochs_reg}, Loss: {loss_reg.item():.4f}")

# 6. 평가
model_reg.eval()
with torch.no_grad():
    predictions_reg = model_reg(X_test_tensor)
    mse_reg = mean_squared_error(y_test_tensor.numpy(), predictions_reg.numpy())
    print(f"\nRegression Test MSE: {mse_reg:.4f}")


[Regression] Epoch 10/100, Loss: 101.4716
[Regression] Epoch 20/100, Loss: 95.3430
[Regression] Epoch 30/100, Loss: 87.6621
[Regression] Epoch 40/100, Loss: 77.5774
[Regression] Epoch 50/100, Loss: 64.9656
[Regression] Epoch 60/100, Loss: 50.6927
[Regression] Epoch 70/100, Loss: 37.1698
[Regression] Epoch 80/100, Loss: 27.7031
[Regression] Epoch 90/100, Loss: 23.3032
[Regression] Epoch 100/100, Loss: 20.8966

Regression Test MSE: 20.0750
